In [1]:
import pandas as pd
import numpy as np

In [2]:
test_df = pd.read_csv('Data/test_clear.csv')
train_df = pd.read_csv('Data/train_clear.csv')

### XGBoost


In [3]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop(['PassengerId'], axis=1).copy()

In [4]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

parameters = {
    'max_depth': [3, 4, 5, 6, 7, 8], 
    'n_estimators': [5, 10, 20, 50, 100],
    'learning_rate': np.linspace(0.02,0.16,8)
}

xgb_model = GridSearchCV(xgb.XGBClassifier(), parameters, cv=5).fit(X_train, Y_train)

print(accuracy_score(Y_train, xgb_model.predict(X_train)))
print(xgb_model.best_score_)
print(xgb_model.best_params_)
print(xgb_model.best_estimator_)

acc_xgb = round(xgb_model.score(X_train, Y_train) * 100, 2)
print(acc_xgb)

0.835016835016835
0.8316238779737619
{'learning_rate': 0.04, 'max_depth': 3, 'n_estimators': 20}
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.04, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=20, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
83.5


In [5]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": xgb_model.predict(X_test)
    })
submission.to_csv('titanic-solution-xgBoost.csv', index=False)